In [ ]:
######################################################################

# I acknowledge the use of AI tools in debugging and writing the code

######################################################################

In [1]:
import pandas as pd
from urllib.request import urlopen
import certifi
import json
import time
from concurrent.futures import ThreadPoolExecutor

# Function to fetch and parse JSON data from a URL
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

# Function to get the S&P 500 tickers list
def get_sp500_tickers():
    sp500_url = "https://financialmodelingprep.com/api/v3/api_key"
    tickers_data = get_jsonparsed_data(sp500_url)
    return ["NIO"]
#     return [item['symbol'] for item in tickers_data]  

# Function to fetch data for a specific ticker, year, and quarter
def fetch_data(ticker, year, quarter, base_url, apikey):
    url = f"{base_url}{ticker}?year={year}&quarter={quarter}&apikey={apikey}"
    try:
        data = get_jsonparsed_data(url)
        return [
            {
                "ticker": ticker,
                "year": year,
                "quarter": quarter,
                "date": transcript.get("date"),
                "content": transcript.get("content")
            }
            for transcript in data
        ]
    except Exception as e:
        print(f"Error fetching data for {ticker} in {year} Q{quarter}: {e}")
        return []

    

# Main function to handle parallel API requests
def main(years):
    # Base URL for the earning call transcripts API
    base_url = "https://financialmodelingprep.com/api/v3/earning_call_transcript/"
    apikey = "api_key"

#     # Define the range of years and quarters
#     years = [2024, 2025]
#              #2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
    quarters = [1, 2, 3, 4]

    # Get the list of all S&P 500 tickers
    tickers = get_sp500_tickers()

    # List to store all data
    all_data = []
    i = 0
    # Use ThreadPoolExecutor for parallel requests
    with ThreadPoolExecutor(max_workers=15) as executor:
        futures = []
        for ticker in tickers:
            for year in years:
                for quarter in quarters:
                    i+=1
                    if (i + 1) % 10 == 0:  # Pause for 1 second every 15 transcripts
                        time.sleep(1)
                        print(i)
                    futures.append(executor.submit(fetch_data, ticker, year, quarter, base_url, apikey))

        # Collect results and pause every 15 transcripts
        for i, future in enumerate(futures):
            all_data.extend(future.result())


    # Convert the list to a DataFrame
    df = pd.DataFrame(all_data)

    # Display the DataFrame
    return df

# Run the main function



In [2]:
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
df_NIO = main(years)

/var/folders/m2/ny_3r2zn14z8qxzlqbt2q1dh0000gn/T/ipykernel_19708/2526519687.py:10: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


9
19
29
39


In [3]:
df_NIO.columns

Index(['ticker', 'year', 'quarter', 'date', 'content'], dtype='object')

In [4]:
# import pandas as pd
# import re

# # Ensure the date column is datetime
# df_NIO['date'] = pd.to_datetime(df_NIO['date'])


# # Filter df_2026 for these tickers
# # df_2026 = df_2026[df_2026['ticker'].isin(peer_tickers)]
# # Function to analyze keyword mentions
# def transcripts_with_keyword(df, keyword, after_date):
#     filtered_df = df[df['date'] > after_date].copy()

#     # Count occurrences of the keyword (case-insensitive)
#     filtered_df['keyword_count'] = filtered_df['content'].str.count(re.escape(keyword), flags=re.IGNORECASE)

#     # Filter transcripts where keyword is mentioned at least once
#     result_df = filtered_df[filtered_df['keyword_count'] > 0]

#     return result_df[['ticker', 'date', 'keyword_count', 'content']]


# # Example usage (easy to change keyword and date)
# keyword = 'tariff'
# after_date = '2025-01-01'

# results = transcripts_with_keyword(df_NIO, keyword, after_date)

# # Display results
# for idx, row in results.iterrows():
#     print(f"\nTicker: {row['ticker']} | Date: {row['date'].date()} | '{keyword}' mentioned {row['keyword_count']} times")
#     print("-" * 80)
# #     print(row['content'][:500] + '...')  # printing first 500 chars for brevity

# # Percentage calculation
# percentage = len(results) / len(df_NIO[df_NIO['date'] > after_date]) * 100
# print(f"\n{percentage:.2f}% of transcripts after {after_date} mention '{keyword}'.")

ZeroDivisionError: division by zero

In [16]:
df_alls['date'] = pd.to_datetime(df_alls['date'])  # note: pd.to_datetime, not pd.to_date_time
# df_alls[df_alls['date'].dt.year == 2024]

df_filtered = df_alls[df_alls['date'] < pd.to_datetime('2024-02-23')]
print(df_filtered) 

# print(df_alls[df_alls['year'] == 2025])  

     ticker  year  quarter                date  \
8      WDAY  2024        1 2023-05-25 16:30:00   
9      WDAY  2024        2 2023-08-24 16:30:00   
10     WDAY  2024        3 2023-11-28 20:25:11   
19     DELL  2024        1 2023-06-01 16:30:00   
20     DELL  2024        2 2023-08-31 16:30:00   
...     ...   ...      ...                 ...   
1927     DE  2024        1 2024-02-15 10:00:00   
1962    GIS  2024        1 2023-09-20 09:00:00   
1963    GIS  2024        2 2023-12-20 09:00:00   
2046     PG  2024        1 2023-10-18 08:30:00   
2047     PG  2024        2 2024-01-23 08:30:00   

                                                content  
8     Operator: Welcome to Workday's First Quarter F...  
9     Operator: Welcome to Workday's Fiscal 2024 Sec...  
10    Operator: Welcome to Workday's Fiscal 2024 Thi...  
19    Operator: Good afternoon, and welcome to the f...  
20    Operator: Good afternoon, and welcome to the F...  
...                                                

In [41]:
# df_2026['date'] = pd.to_datetime(df_2026['date'])

# print(df_2026[(df_2026['date'].dt.year >= 2025) & (df_2026['date'].dt.month >= 3)])
df_2026 = df_2026[df_2026['date'].dt.year >= 202]
print(df_2026)

     ticker  year quarter                date  \
3       APO  2024       4 2025-02-04 08:30:00   
7       LII  2024       4 2025-01-29 09:30:00   
15     WDAY  2025       4 2025-02-25 16:30:00   
19      TPL  2024       4 2025-02-20 08:30:00   
27     DELL  2025       4 2025-02-28 17:26:00   
...     ...   ...     ...                 ...   
2138     SO  2024       4 2025-02-20 13:00:00   
2142   SPGI  2024       4 2025-02-11 08:30:00   
2146    UNP  2024       4 2025-01-23 08:45:00   
2150    XEL  2024       4 2025-02-06 10:00:00   
2154    XOM  2024       4 2025-01-31 03:30:00   

                                                content  
3     Operator: Good morning, and welcome to Apollo ...  
7     Operator: Welcome to the Lennox Fourth Quarter...  
15    Operator: Welcome to Workday's Fourth Quarter ...  
19    Operator: Greetings, and welcome to the Texas ...  
27    Operator: Good afternoon and welcome to the Fi...  
...                                                 ...  
2138 

In [44]:
# df_old['year'] = df_old['year'].astype(str)
# df_old['quarter'] = df_old['quarter'].astype(str)
# df_2026['year'] = df_2026['year'].astype(str)
# df_2026['quarter'] = df_2026['quarter'].astype(str)

# # Remove rows from df_old that match ticker/year/quarter in df_2026
# df_old_filtered = df_old[~df_old.set_index(['ticker', 'year', 'quarter']).index.isin(
#     df_2026.set_index(['ticker', 'year', 'quarter']).index
# )]

# # Concatenate df_old (filtered) with df_2026
# merged_df = pd.concat([df_old_filtered, df_2026], ignore_index=True)

# Result
print(merged_df)

merged_df = merged_df.to_csv("/Users/marcosmaldacena/Downloads/Thesis Data/transcripts_2006_2025_03_18.csv")

       Unnamed: 0 ticker  year quarter                 date  \
0             0.0   DELL  2005       4  2006-01-26 11:39:13   
1             1.0   DELL  2006       1  2006-04-20 12:46:09   
2             2.0   DELL  2006       2  2006-07-14 12:00:43   
3             3.0   DELL  2006       3  2006-10-17 13:18:26   
4             4.0   DELL  2006       4  2007-01-23 12:15:14   
...           ...    ...   ...     ...                  ...   
31468         NaN     SO  2024       4  2025-02-20 13:00:00   
31469         NaN   SPGI  2024       4  2025-02-11 08:30:00   
31470         NaN    UNP  2024       4  2025-01-23 08:45:00   
31471         NaN    XEL  2024       4  2025-02-06 10:00:00   
31472         NaN    XOM  2024       4  2025-01-31 03:30:00   

                                                 content  
0      Executives: Tony Takazawa, Vice President of I...  
1      Executives: Tony Takazawa, Vice President, Inv...  
2      Executives: Joe Tucci - Chairman, President, C...  
3      

In [36]:
# file_path = "/Users/marcosmaldacena/Downloads/Thesis Data/updated_2024_2025.csv"
# df_2026.to_csv(file_path)


df_old = pd.read_csv("/Users/marcosmaldacena/Downloads/Thesis Data/transcripts_2006_2025.csv")
print(df_old)




       Unnamed: 0 ticker  year  quarter                 date  \
0             0.0   DELL  2005        4  2006-01-26 11:39:13   
1             1.0   DELL  2006        1  2006-04-20 12:46:09   
2             2.0   DELL  2006        2  2006-07-14 12:00:43   
3             3.0   DELL  2006        3  2006-10-17 13:18:26   
4             4.0   DELL  2006        4  2007-01-23 12:15:14   
...           ...    ...   ...      ...                  ...   
30997         NaN    XEL  2023        4  2024-01-25 00:00:00   
30998         NaN    XOM  2023        1  2023-04-28 09:51:04   
30999         NaN    XOM  2023        2  2023-07-28 10:52:10   
31000         NaN    XOM  2023        3  2023-10-27 11:40:16   
31001         NaN    XOM  2023        4  2024-02-02 12:02:10   

                                                 content  
0      Executives: Tony Takazawa, Vice President of I...  
1      Executives: Tony Takazawa, Vice President, Inv...  
2      Executives: Joe Tucci - Chairman, President, C.

In [4]:
import pandas as pd

# File path
file_path = "/Users/marcosmaldacena/Downloads/2005_2009.csv"


file_path_2 = "/Users/marcosmaldacena/Downloads/2009_2013.csv"


# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows
print(df.head())  # Shows the first 5 rows by default

# Optionally, display all column names and the shape of the dataset
print(df['year'])

   Unnamed: 0 ticker  year  quarter                 date  \
0           0   DELL  2005        4  2006-01-26 11:39:13   
1           1   DELL  2006        1  2006-04-20 12:46:09   
2           2   DELL  2006        2  2006-07-14 12:00:43   
3           3   DELL  2006        3  2006-10-17 13:18:26   
4           4   DELL  2006        4  2007-01-23 12:15:14   

                                             content  
0  Executives: Tony Takazawa, Vice President of I...  
1  Executives: Tony Takazawa, Vice President, Inv...  
2  Executives: Joe Tucci - Chairman, President, C...  
3  Executives: Joe Tucci - Chairman, President, C...  
4  TRANSCRIPT SPONSOR :\nExecutives: Tony Takazaw...  
0       2005
1       2006
2       2006
3       2006
4       2006
        ... 
4394    2008
4395    2009
4396    2009
4397    2009
4398    2009
Name: year, Length: 4399, dtype: int64


In [7]:
import pandas as pd

# File paths
file_path_1 = "/Users/marcosmaldacena/Downloads/2005_2009.csv"
file_path_2 = "/Users/marcosmaldacena/Downloads/2009_2013.csv"

# Load the CSV files into DataFrames
df1 = pd.read_csv(file_path_1)
df2 = pd.read_csv(file_path_2)

# Combine the two DataFrames
combined_df = pd.concat([df1, df2], ignore_index=True)

# Remove duplicates based on the 'content' column
if 'content' in combined_df.columns:
    combined_df = combined_df.drop_duplicates(subset=['content'])
else:
    print("The 'content' column is not found in the dataset. No duplicates removed based on 'content'.")

# Display the first few rows of the combined DataFrame
print("First few rows of the combined DataFrame after removing duplicates by 'content':")
print(combined_df.head())

# Display the shape of the combined DataFrame
print("\nShape of the combined DataFrame after removing duplicates by 'content':", combined_df.shape)

# Display column names
print("\nColumns in the combined DataFrame:")
print(combined_df.columns)

# Display summary statistics
print("\nSummary Statistics:")
print(combined_df.describe())

# Display unique years and their counts
if 'year' in combined_df.columns:
    print("\nYear-wise counts:")
    print(combined_df['year'].value_counts())
else:
    print("\nThe 'year' column is not present in the dataset.")

    


First few rows of the combined DataFrame after removing duplicates by 'content':
   Unnamed: 0 ticker  year  quarter                 date  \
0           0   DELL  2005        4  2006-01-26 11:39:13   
1           1   DELL  2006        1  2006-04-20 12:46:09   
2           2   DELL  2006        2  2006-07-14 12:00:43   
3           3   DELL  2006        3  2006-10-17 13:18:26   
4           4   DELL  2006        4  2007-01-23 12:15:14   

                                             content  
0  Executives: Tony Takazawa, Vice President of I...  
1  Executives: Tony Takazawa, Vice President, Inv...  
2  Executives: Joe Tucci - Chairman, President, C...  
3  Executives: Joe Tucci - Chairman, President, C...  
4  TRANSCRIPT SPONSOR :\nExecutives: Tony Takazaw...  

Shape of the combined DataFrame after removing duplicates by 'content': (10520, 6)

Columns in the combined DataFrame:
Index(['Unnamed: 0', 'ticker', 'year', 'quarter', 'date', 'content'], dtype='object')

Summary Statistics:
 

In [12]:
import pandas as pd

# File paths
file_path_1 = "/Users/marcosmaldacena/Downloads/2005_2009.csv"
file_path_2 = "/Users/marcosmaldacena/Downloads/2009_2013.csv"
file_path_merged = "/Users/marcosmaldacena/Downloads/Thesis Data/merged_earnings_call_transcripts.csv"
output_path = "/Users/marcosmaldacena/Downloads/Thesis Data/transcripts_2006_2025.csv"

# Load the first two CSV files
df1 = pd.read_csv(file_path_1)
df2 = pd.read_csv(file_path_2)

# Combine the first two DataFrames
combined_df = pd.concat([df1, df2], ignore_index=True)

# Remove duplicates based on the 'content' column
if 'content' in combined_df.columns:
    combined_df = combined_df.drop_duplicates(subset=['content'])
else:
    print("The 'content' column is not found in the dataset. No duplicates removed based on 'content'.")

# Load the merged DataFrame
df_merged = pd.read_csv(file_path_merged)

# Merge the combined DataFrame with the merged DataFrame
final_df = pd.concat([combined_df, df_merged], ignore_index=True)

# Remove duplicates from the final DataFrame based on the 'content' column
if 'content' in final_df.columns:
    final_df = final_df.drop_duplicates(subset=['content'])
else:
    print("The 'content' column is not found in the final dataset. No duplicates removed based on 'content'.")

# Save the resulting DataFrame to a new CSV file
final_df.to_csv(output_path, index=False)

# Display the first few rows of the saved DataFrame
print("First few rows of the final DataFrame:")
print(final_df.head())

# Display the shape of the final DataFrame
print("\nShape of the final DataFrame:", final_df.shape)

# Display summary statistics
print("\nSummary Statistics:")
print(final_df.describe())

# Display unique years and their counts
if 'year' in final_df.columns:
    print("\nYear-wise counts:")
    print(final_df['year'].value_counts())
else:
    print("\nThe 'year' column is not present in the final dataset.")


First few rows of the final DataFrame:
   Unnamed: 0 ticker  year  quarter                 date  \
0         0.0   DELL  2005        4  2006-01-26 11:39:13   
1         1.0   DELL  2006        1  2006-04-20 12:46:09   
2         2.0   DELL  2006        2  2006-07-14 12:00:43   
3         3.0   DELL  2006        3  2006-10-17 13:18:26   
4         4.0   DELL  2006        4  2007-01-23 12:15:14   

                                             content  
0  Executives: Tony Takazawa, Vice President of I...  
1  Executives: Tony Takazawa, Vice President, Inv...  
2  Executives: Joe Tucci - Chairman, President, C...  
3  Executives: Joe Tucci - Chairman, President, C...  
4  TRANSCRIPT SPONSOR :\nExecutives: Tony Takazaw...  

Shape of the final DataFrame: (31002, 6)

Summary Statistics:
         Unnamed: 0          year       quarter
count  10520.000000  31002.000000  31002.000000
mean    3160.687262   2015.996226      2.503032
std     2068.115831      5.053530      1.114088
min        0.00

In [3]:
df_2013 = pd.read_csv("/Users/marcosmaldacena/Downloads/Thesis Data/Earnings Calls/sp500_earnings_transcripts.csv")
df_2014 = pd.read_csv("/Users/marcosmaldacena/Downloads/Thesis Data/2014_earnings_call_transcripts.csv")
df_2015_2016 = pd.read_csv("/Users/marcosmaldacena/Downloads/Thesis Data/2015_2016_earnings_call_transcripts.csv")
df_others = pd.read_csv("/Users/marcosmaldacena/Downloads/Thesis Data/other_years_earnings_call_transcripts.csv")
df_2023 = pd.read_csv("/Users/marcosmaldacena/Downloads/Thesis Data/Earnings Calls/sp500_earnings_transcripts_2023.csv")





In [5]:
print(df_2023)

     ticker  year  quarter                 date  \
0      DELL  2023        1  2022-05-26 21:37:11   
1      DELL  2023        2  2022-08-25 23:12:03   
2      DELL  2023        3  2022-11-21 19:34:06   
3      DELL  2023        4  2023-03-02 20:44:03   
4      ERIE  2023        1  2023-04-28 14:44:15   
...     ...   ...      ...                  ...   
1961    XEL  2023        4  2024-01-25 00:00:00   
1962    XOM  2023        1  2023-04-28 09:51:04   
1963    XOM  2023        2  2023-07-28 10:52:10   
1964    XOM  2023        3  2023-10-27 11:40:16   
1965    XOM  2023        4  2024-02-02 12:02:10   

                                                content  
0     Operator: Good afternoon, and welcome to the F...  
1     Operator: Please stand by. Good afternoon. And...  
2     Operator: Good afternoon, ladies and gentlemen...  
3     Operator: Good afternoon, and welcome to the F...  
4     Operator: Good morning, and welcome to the Eri...  
...                                    

In [8]:
!llama model list

+-----------------------------------------+-----------------------------------------------------+----------------+
| Model Descriptor                        | Hugging Face Repo                                   | Context Length |
+-----------------------------------------+-----------------------------------------------------+----------------+
| Llama3.1-8B                             | meta-llama/Llama-3.1-8B                             | 128K           |
+-----------------------------------------+-----------------------------------------------------+----------------+
| Llama3.1-70B                            | meta-llama/Llama-3.1-70B                            | 128K           |
+-----------------------------------------+-----------------------------------------------------+----------------+
| Llama3.1-405B:bf16-mp8                  | meta-llama/Llama-3.1-405B                           | 128K           |
+-----------------------------------------+-------------------------------------

In [ ]:
!llama model download --source meta --model-id Llama3.1-70B-Instruct --url "https://llama3-1.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoiMWdnbmRodTVuMno2bzhjNXUzbWdqdWZvIiwiUmVzb3VyY2UiOiJodHRwczpcL1wvbGxhbWEzLTEubGxhbWFtZXRhLm5ldFwvKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTMzODg0N319fV19&Signature=rY2tZefAQ7GxLjm5nc4koK7EQ4H48j-anoWyr7r33FKTTVcGrY2MdyAZhfPceMbPrNAtekYwTExWIrDk67ug1YpQUXWoFJU9-ojBWi0TVVn-8ye4%7EIqTZhjfnbkFdMw38tjGM1EBLdcueNt46dYJoDokyp%7EE%7EWxum7nU4cw4q1KE9ShuY0Yrh5uMVR5GEKHpVhHP4J8lksBb%7ErBDA0j5Kc7zt2KaGgd-RbS-H%7EtPPpNy%7ElFnFlNgY0-UXYBt8IuCvc2jQggy1JGyN%7Egt4QxUk8Xl5paceiYJBmsqt6XCAswlqCCI4Ls6LVGxdvEV3TkBTc0cYSdiM%7EzZVjQ53YEUrg__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1242721713647035"


In [13]:
"https://llama3-1.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOiI7InVuaXF1ZV9oYXNoIjoibmRodTVuMno2bzhjNXUzbWdqdWZvIiwidHRwczovLmxsYW1hLm1ldGEubmV0Ly...&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1242721"


'https://llama3-1.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOiI7InVuaXF1ZV9oYXNoIjoibmRodTVuMno2bzhjNXUzbWdqdWZvIiwidHRwczovLmxsYW1hLm1ldGEubmV0Ly...&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1242721'